In [12]:
import torch
import requests
import os
from pathlib import Path
URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [11]:
r = requests.get(URL)


with open('../resnet18-5c106cde.pth', 'wb') as f:
    f.write(r.content)

In [29]:
import io
from utils.ModuleTransfer import ModuleTransfer
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152
from nn.models.classification.resnet import ResNet


zoo = {
    resnet18: 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    resnet34: 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    resnet50: 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    resnet101: 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    resnet152: 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

zoo_models_mapping = {
    
    resnet18: ResNet.resnet18,
    resnet34: ResNet.resnet34,
    resnet50: ResNet.resnet50,
    resnet101: ResNet.resnet101,
    resnet152: ResNet.resnet152,
}

for src_def, dst_def in zoo_models_mapping.items():
    url = zoo[src_def]
    print(f"Getting weights at url={url}")
    
    r = requests.get(url)
    
    with open('../temp.pth', 'wb') as f:
        f.write(r.content)

    src = src_def(False)
    src.load_state_dict(torch.load('../temp.pth'))
    dst = dst_def()

    x = torch.rand((1, 3, 224, 224))

    src.eval()
    dst.eval()
    
    a = src(x)
    b = dst(x)

    assert not torch.equal(a, b)
    
    
    ModuleTransfer(src, dst)(x)

    a = src(x)
    b = dst(x)

    assert torch.equal(a, b)

Getting weights at url=https://download.pytorch.org/models/resnet18-5c106cde.pth
Getting weights at url=https://download.pytorch.org/models/resnet34-333f7ec4.pth
Getting weights at url=https://download.pytorch.org/models/resnet50-19c8e357.pth
Getting weights at url=https://download.pytorch.org/models/resnet101-5d3b4d8f.pth
Getting weights at url=https://download.pytorch.org/models/resnet152-b121ed2d.pth


In [ ]:
!ls